In [ ]:
# installation paths
# !pip install 

In [3]:
# import dependencies
import zipfile
import os.path
import pandas as pd
import numpy as np

In [4]:
import Group5FileLib as fl


# Unzip files if they are not unzipped on local machine
fl.UnzipFiles()

# Combine Zipped Files
viewFile = fl.CombineFiles()

# Load Lot Fact File
lotFile = fl.LoadLotFacts()

File exists data/VCardTransactions-NoName.csv - NOT un-zipping
File exists data/VCardTransactions-NoName2.csv - NOT un-zipping
File exists data/vtrans.csv - NOT merging


In [5]:
# read files into a data frame
vtrans = pd.read_csv(viewFile)
capacity = pd.read_csv(lotFile)


In [6]:
# read lot facts csv file into pandas dataframe
capacity = capacity.sort_values(by="lot", ascending=True)
capacity = capacity.reset_index(drop=True)
capacity

,lot,capacity
0,2,34
1,3,217
2,4,48
3,6,20
4,7,10
5,8,37
6,9,18
7,10,462
8,11,591
9,12,88


In [ ]:
# return vtrans dataframe
vtrans

In [ ]:
# col_titles = ['tTransID', 'CardNumber', 'LotNumber', 'EffectiveGroupName', 'EntranceTime',
#         'ExitTime', 'NoEntry', 'noexit']

# vtrans = vtrans.reindex(columns=col_titles)
# vtrans

In [ ]:
# breakout vtrans date/time/day of week into individual columns

vtrans['EntranceDate'] = pd.to_datetime(vtrans['EntranceTime']).dt.date
vtrans['EntranceTime'] = pd.to_datetime(vtrans['EntranceTime']).dt.strftime('%H:%M:%S')
vtrans['ExitDate'] = pd.to_datetime(vtrans['ExitTime']).dt.date
vtrans['ExitTime'] = pd.to_datetime(vtrans['ExitTime']).dt.strftime('%H:%M:%S')

# Sort date determined to not be needed - commenting out
# vtrans['SortDate'] = pd.to_datetime(vtrans['Sort_Time']).dt.date
# vtrans['SortTime'] = pd.to_datetime(vtrans['Sort_Time']).dt.strftime('%H:%M:%S')


In [ ]:
# breakout vtrans day of week into individual column
vtrans['EntranceDay'] = pd.to_datetime(vtrans['EntranceDate']).dt.day_name()
vtrans['ExitDay'] = pd.to_datetime(vtrans['ExitDate']).dt.day_name()

# Sort date determined to not be needed - commenting out
# vtrans['SortDay'] = pd.to_datetime(vtrans['Sort_Time']).dt.day_name()

In [ ]:
# rearrange and organize columns
col_titles = ['tTransID', 'CardNumber', 'LotNumber', 'EffectiveGroupName','EntranceDate', 'EntranceTime', 'EntranceDay',
       'ExitDate', 'ExitTime', 'ExitDay', 'NoEntry', 'noexit']

vtrans = vtrans.reindex(columns=col_titles)
vtrans

In [ ]:
# getting counts for days of week to verify to_day code worked as expected. 
e_count = vtrans.value_counts("EntranceDay")
x_count = vtrans.value_counts("ExitDay")

# show value counts

e_count
# x_count

In [ ]:
# create new data frame to get daily counts by lot and average time on site
# lottime = pd.DataFrame()

# Bringing the lot capacity into the vtrans data frame
lottime = vtrans.merge(lotdata, 
                     left_on ="LotNumber")
lottime

In [ ]:
# Defining a widget to select a certain lot#

from ipywidgets import widgets, interactive, Layout

w_lot = widgets.Dropdown(
    options = ["All"] + sorted(set(vtrans.LotNumber)),     
    value = "All",
    description = 'Lot #:',
    style = {"description_width": '100px'},
    layout = Layout(width="25%"),
    disabled = False
)

def view(lot):
    if lot == "All":
        dftmp = vtrans
    else:
        dftmp = vtrans[vtrans.LotNumber == lot]
    
    display(dftmp)

i = interactive(view, lot=w_lot)
display(i)

In [ ]:
# backfill any n/a values to get estimates of error times

vtrans = vtrans.dropna()

# create time stamps

exit = pd.Timestamp(vtrans["ExitTime"])
entrance = pd.Timestamp(vtrans["EntranceTime"])

vtrans["TimeParked"] = exit - entrance


In [ ]:
# Defining a widget to select data for a certain day of the week

from ipywidgets import widgets, interactive, Layout

w_dow = widgets.RadioButtons(
    options = ["All", "Monday", "Tuesday", "Wednesday", "Thursday","Friday","Saturday","Sunday"],     
    value = "All",
    description = 'Day of the week:',
    style = {"description_width": '100px'},
    layout = Layout(width="25%"),
    disabled = False
)

def view(dow):
    if dow == "All":
        dftmp = vtrans
    else:
        dftmp = vtrans[vtrans.EntranceDay == dow]
    
    display(dftmp)

i = interactive(view, dow=w_dow, view, lot=w_lot)
display(i)

In [ ]:
# Calculating the number of days a car was parked

vtrans['ParkedDays'] = vtrans['ExitDate'] - vtrans['EntranceDate']


In [ ]:
# Calculating the number of hours a car was parked

vtrans["EntryHour"] = vtrans["EntranceTime"].dt.hour
vtrans["ExistHour"] = vtrans["ExitTime"].dt.hour
vtrans["ParkedTime"] = vtrans["ExistHour"] - vtrans["EntryHour"]
vtrans

In [ ]:
# Bringing the lot capacity into the vtrans data frame
   
Left_join = pd.merge(vtrans, 
                     lotdata, 
                     on ='LotNumber', 
                     how ='left')
Left_join

In [ ]:
# We could try a "pseudo" capacity utilization. We could assume each lot capacity in hours is 24 (hrs) x capacity 
# for any given day
# If we can groupby an aggregate function of LotNumber, EntranceDate and ParkedTime we could divide this to the 24 (hrs) x capacity
# does anyone know how to do this aggregation ?


In [ ]:
vtrans